# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# create the file path to read CSV file
f_path = os.path.join("..", "WeatherPy", 'WeatherPy.csv')
city_df = pd.read_csv(f_path)
city_df.head()

#clean data frame, check to see if there is any data missing in each column
city_df.isnull().any()

#count the data in each row
city_df.count()

city_df[city_df.isnull().any(axis=1)]

#drop rows with missing values
new_df = city_df.dropna(how='any')

new_df.count()

new_df.dtypes

city           object
lat           float64
lng           float64
Max Temp      float64
Humidity      float64
Cloudiness    float64
Wind Speed    float64
Country        object
Date          float64
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
#get lats and lngs for the locations 
locations = new_df[['lat', 'lng']]

print(locations)

#get humidity as weight from df
humidity = new_df['Humidity']

humidity.dtype

       lat     lng
0    47.66  -70.16
1    42.60  -73.97
2    35.10   33.42
3    10.57   72.64
4    -0.74  -90.35
..     ...     ...
524  22.85   82.20
525  37.46 -122.43
526  22.48   92.97
527  63.01  112.47
528 -21.69  165.68

[483 rows x 2 columns]


dtype('float64')

In [5]:
#plot maps
fig = gmaps.figure()

#add humidity to gmaps
hum_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius=0.9)

fig.add_layer(hum_layer)

#show map
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#max temp 70 < x < 80
#wind speed less than 10 mpg
#zero cloudiness

#get new DF for temps between 70 and 80 degrees, with less than 10 mph wind speed, and zero cloudiness LOC 
updated_df = new_df[(new_df['Max Temp'] >= 70) & (new_df['Max Temp'] <= 80) & (new_df['Wind Speed'] < 10) & (new_df['Cloudiness'] == 0)]

#set to new dataframe and rename to hotel_df 
hotel_df = pd.DataFrame(updated_df)

hotel_df

,city,lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
110,caravelas,-17.71,-39.25,72.50,82.0,0.0,7.65,BR,1.599869e+09
136,tukrah,32.53,20.58,73.81,79.0,0.0,3.98,LY,1.599869e+09
137,behshahr,36.69,53.55,70.50,61.0,0.0,2.84,IR,1.599869e+09
239,ardakan,32.31,54.02,77.00,22.0,0.0,2.24,IR,1.599869e+09
324,biograd na moru,43.94,15.45,73.00,68.0,0.0,6.93,HR,1.599869e+09
356,sao francisco,-15.95,-44.86,74.80,25.0,0.0,6.80,BR,1.599869e+09
381,ituiutaba,-18.97,-49.47,79.05,19.0,0.0,5.77,BR,1.599869e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# create a new column for hotel name
hotel_df['Hotel Name'] = ""

#print new dataframe 
hotel_df


,city,lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
110,caravelas,-17.71,-39.25,72.50,82.0,0.0,7.65,BR,1.599869e+09,
136,tukrah,32.53,20.58,73.81,79.0,0.0,3.98,LY,1.599869e+09,
137,behshahr,36.69,53.55,70.50,61.0,0.0,2.84,IR,1.599869e+09,
239,ardakan,32.31,54.02,77.00,22.0,0.0,2.24,IR,1.599869e+09,
324,biograd na moru,43.94,15.45,73.00,68.0,0.0,6.93,HR,1.599869e+09,
356,sao francisco,-15.95,-44.86,74.80,25.0,0.0,6.80,BR,1.599869e+09,
381,ituiutaba,-18.97,-49.47,79.05,19.0,0.0,5.77,BR,1.599869e+09,


In [8]:
# set paramters for API to search 
params = {
    'radius': 5000,
    'keyword': "hotels",
    'types': 'hotel',
    'key': g_key
}

#create a loop with iterrows to call API and search for closest Hotel 
for index, row in hotel_df.iterrows():
    lat = row['lat']
    lng = row['lng']
    
    params['location'] = f'{lat},{lng}'
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(base_url, params=params).json()
   
    results = response['results']
    
#     pprint(results)

    #put in a try/except bc i got a key error the first time i tried it and then an index error the second time i tried it
    try:
        
        #print hotel names into the dataframe 
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']

    except (KeyError, IndexError):
        print('No hotel found, skipped')


        
# print(hotels)


No hotel found, skipped


In [9]:
#print new hotel df to check if hotel printed
hotel_df

,city,lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
110,caravelas,-17.71,-39.25,72.50,82.0,0.0,7.65,BR,1.599869e+09,Pousada Encanto Abrolhos
136,tukrah,32.53,20.58,73.81,79.0,0.0,3.98,LY,1.599869e+09,
137,behshahr,36.69,53.55,70.50,61.0,0.0,2.84,IR,1.599869e+09,Ashraf Hotel
239,ardakan,32.31,54.02,77.00,22.0,0.0,2.24,IR,1.599869e+09,Kheshtomah Ecolodge
324,biograd na moru,43.94,15.45,73.00,68.0,0.0,6.93,HR,1.599869e+09,Villas Garamm Resort
356,sao francisco,-15.95,-44.86,74.80,25.0,0.0,6.80,BR,1.599869e+09,Hotel Avenida - São FRancisco MG
381,ituiutaba,-18.97,-49.47,79.05,19.0,0.0,5.77,BR,1.599869e+09,Lider Hotel


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))